In [382]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import missingno as msno

players = pd.read_csv('./Dataset/players.csv')
teams = pd.read_csv('./Dataset/teams.csv')
players_teams = pd.read_csv('./Dataset/players_teams.csv')
awards_players = pd.read_csv('./Dataset/awards_players.csv')
coaches = pd.read_csv('./Dataset/coaches.csv')
series_post = pd.read_csv('./Dataset/series_post.csv')
teams_post = pd.read_csv('./Dataset/teams_post.csv')


### Awards_players


- corrected name of a prize
- drop lgID
- calculate the number of prizes that each player has (having in consideration the year)
- drop award (name)

In [383]:
awards_players['award'] = awards_players['award'].replace("Kim Perrot Sportsmanship", "Kim Perrot Sportsmanship Award")
awards_players = awards_players.drop(columns=['lgID'], axis=1)

award_weights = {
    "All-Star Game Most Valuable Player": 2,
    "Coach of the Year": 3,
    "Defensive Player of the Year": 4,
    "Kim Perrot Sportsmanship Award": 1,
    "Most Improved Player": 2,
    "Most valuable Player": 5,
    "Rookie of the Year": 3,
    "Sixth Woman of the Year": 2,
    "WNBA Finals Most Valuable Player": 4,
    "WNBA All-Decade Team": 1,
    "WNBA All Decade Team Honorable Mention": 1,
}

# Sort the DataFrame by playerID and year
awards_players.sort_values(by=["playerID", "year"], inplace=True)

awards_players['award_weight'] = awards_players['award'].map(award_weights)
awards_players['num_prizes'] = awards_players.groupby('playerID')['award_weight'].cumsum()

awards_players = awards_players.drop(columns=['award', 'award_weight'], axis=1)

awards_players.to_csv('./Modified/modified_awards.csv', index=False)


### Players

- drop college, collegeOther, pos and lastseason
- remove who has height = 0
- replace height error (9) with the average
- replace weight error (0) with the average


In [384]:
players = players.drop('college', axis=1)
players = players.drop('pos', axis=1)
players = players.drop('collegeOther', axis=1)
# Have only 0s
players = players.drop('lastseason', axis=1)
players = players.drop('firstseason', axis=1)

players = players[players['height'] > 0]

# Calculate the mean of non-zero heights
mean_height = players[(players['height'] != 0) | (players['height'] != 9)]['height'].mean()
# Replace 9 with the mean height
players['height'] = players['height'].replace(9, mean_height)

# Calculate the mean of non-zero weights
mean_weight = players[players['weight'] != 0]['weight'].mean()
players['weight'] = players['weight'].replace(0, mean_weight)


### Players_teams

- statistics: mean of the last two years
    - if just appeared 1 year before, use the info of that year
    - if never appeared before, null ( we doesnt know anything about that player at the beginning of the season)

In [385]:
players_teams = players_teams.drop(columns=['lgID', 'PostGP','PostGS','PostMinutes','PostPoints','PostoRebounds','PostdRebounds','PostRebounds','PostAssists','PostSteals','PostBlocks','PostTurnovers','PostPF','PostfgAttempted','PostfgMade','PostftAttempted','PostftMade','PostthreeAttempted','PostthreeMade','PostDQ'], axis=1)

players_teams.sort_values(by=["playerID", "year"])

mean_columns = ['GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq']

# Calculate the mean for each column over the rolling window
for col in mean_columns:
    players_teams[f'mean_{col}'] = players_teams.groupby('playerID')[col].shift(1)
    
#players_teams[f'mean_{col}'] = players_teams.groupby('playerID')[col].shift(1).rolling(window=2, min_periods=1).mean().reset_index(level=0, drop=True)


players_teams.drop(mean_columns, axis=1, inplace=True)

- add a column called Age, considering the year of the season. Calculated through the merge between two datasets
- drop birthdate and deathdate


In [386]:
def calculate_age(row):
    birth_year = int(row['birthDate'][:4])
    age = 2000 + row['year'] - birth_year
    return age


merged_data = players.merge(players_teams, left_on='bioID', right_on='playerID')
merged_data = merged_data.merge(awards_players, left_on=['bioID', 'year'], right_on=['playerID', 'year'], how='left')
merged_data.sort_values(by=["bioID", "year"], inplace=True)

merged_data['num_prizes'] = merged_data.groupby('bioID')['num_prizes'].ffill()

# Fill NaN values with 0 for players with no prizes 
merged_data['num_prizes'].fillna(0, inplace=True)

# Apply the function to each row
merged_data['age'] = merged_data.apply(calculate_age, axis=1)

merged_data = merged_data.drop('birthDate', axis=1)
merged_data = merged_data.drop('deathDate', axis=1)
merged_data = merged_data.drop('playerID_x', axis=1)
merged_data = merged_data.drop('playerID_y', axis=1)


### Coaches

- drop lgID, stint, post_wins and post_losses
- create column num_prizes (merging awards_players)

In [387]:
coaches = coaches.drop(columns=['lgID'], axis=1)
coaches = coaches.drop(columns=['stint'], axis=1)
coaches = coaches.drop(columns=['post_wins'], axis=1)
coaches = coaches.drop(columns=['post_losses'], axis=1)

coaches = coaches.merge(awards_players, left_on=['coachID', 'year'], right_on=['playerID', 'year'], how='left')
coaches.sort_values(by=["coachID", "year"], inplace=True)

coaches['num_prizes'] = coaches.groupby('coachID')['num_prizes'].ffill()

# Fill NaN values with 0 for players with no prizes 
coaches['num_prizes'].fillna(0, inplace=True)

coaches = coaches.drop(columns=['playerID'], axis=1)


### Teams

- drop seeded, confID, divID, franchID, arena, lgID and name
- corrected error: one season it was registered two winners

In [388]:
teams = teams.drop(columns=['seeded', 'divID', 'franchID', 'arena', 'lgID', 'name', 'tmORB'], axis=1)
teams = teams.drop(columns=['tmDRB'], axis=1)
teams = teams.drop(columns=['tmTRB'], axis=1)
teams = teams.drop(columns=['opptmORB'], axis=1)
teams = teams.drop(columns=['opptmDRB'], axis=1)
teams = teams.drop(columns=['opptmTRB'], axis=1)
teams = teams.drop(columns=['firstRound'], axis=1)
teams = teams.drop(columns=['semis'], axis=1)
teams = teams.drop(columns=['finals'], axis=1)
teams = teams.drop(columns=['o_fgm','o_fga','o_ftm','o_fta','o_3pm','o_3pa','o_oreb','o_dreb','o_reb','o_asts','o_pf','o_stl','o_to','o_blk','o_pts','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts','GP','homeW','homeL','awayW','awayL','confW','confL','min'], axis=1)



columns_to_shift = ['won', 'lost', 'attend', 'rank']
""" 
teams.sort_values(by=["tmID", "year"])
# Group by team and shift the specified columns to get the previous year's values
for col in columns_to_shift:
    teams[f'prev_{col}'] = teams.groupby('tmID')[col].shift(1)

teams = teams[teams.groupby('tmID').cumcount() > 0] """
teams = teams.drop(columns=columns_to_shift, axis=1)

# In the year 6, a victory was incorrectly recorded.
#teams.loc[(teams['year'] == 6) & (teams['tmID'] == 'CON'), 'finals'] = 'L'


- calculate the number of prizes a team has (total of prizes the players have), creating the column num_prizes

In [389]:
merged_data.sort_values(by=["bioID", "year"])

teams.sort_values(by=["tmID", "year"])

playerAndTeams = merged_data.merge(teams, on=["tmID", "year"])

# Group by team and year, summing the number of prizes
team_prizes = playerAndTeams.groupby(['tmID', 'year'])['num_prizes'].sum().reset_index()

# Sort the result by year
team_prizes.sort_values(by='year', inplace=True)

teams = teams.merge(team_prizes, on=["tmID", "year"])


- calculate the sum of the statistics of the players of each team

In [390]:

# Define the columns to sum
sum_columns = ['mean_GP', 'mean_GS', 'mean_minutes', 'mean_points', 'mean_oRebounds', 'mean_dRebounds', 'mean_rebounds', 'mean_assists', 'mean_steals', 'mean_blocks', 'mean_turnovers', 'mean_PF', 'mean_fgAttempted', 'mean_fgMade', 'mean_ftAttempted', 'mean_ftMade', 'mean_threeAttempted', 'mean_threeMade', 'mean_dq']

# Sum the columns for each team and year
summed_columns = playerAndTeams.groupby(['tmID', 'year'])[sum_columns].sum().reset_index()

summed_columns.columns = ['tmID', 'year'] + ['sum_' + col for col in sum_columns]

# Merge the summed columns back into the 'teams' table
teams = pd.merge(teams, summed_columns, how='left', on=['tmID', 'year'])


- calculate the mean of the height, weight and age of the players of each team

In [391]:

team_means = playerAndTeams.groupby(['tmID', 'year']).agg({
    'height': 'mean',
    'weight': 'mean',
    'age': 'mean',
}).reset_index()
teams = pd.merge(teams, team_means, on=['tmID', 'year'], how='left')



- add to the table teams the number of prizes the coach has (sum it in the column num_prizes)

In [392]:
""" teams = pd.merge(teams, coaches[['tmID', 'year', 'won', 'lost']], on=['tmID', 'year'], how='left')

teams = teams.rename(columns={'won_y': 'coach_won', 'lost_y': 'coach_lost'})

print(teams) """

""" merged_df = pd.merge(teams, coaches, on=['year', 'tmID'], how='left')

merged_df['num_prizes_y'] = merged_df['num_prizes_y'].fillna(0)

merged_df['num_prizes'] = merged_df['num_prizes_x'] + merged_df['num_prizes_y']

merged_df = merged_df.drop(['num_prizes_x', 'num_prizes_y'], axis=1)

# Update the 'num_prizes' column in the original teams DataFrame
teams['num_prizes'] = merged_df['num_prizes']

 """

" merged_df = pd.merge(teams, coaches, on=['year', 'tmID'], how='left')\n\nmerged_df['num_prizes_y'] = merged_df['num_prizes_y'].fillna(0)\n\nmerged_df['num_prizes'] = merged_df['num_prizes_x'] + merged_df['num_prizes_y']\n\nmerged_df = merged_df.drop(['num_prizes_x', 'num_prizes_y'], axis=1)\n\n# Update the 'num_prizes' column in the original teams DataFrame\nteams['num_prizes'] = merged_df['num_prizes']\n\n "

In [393]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'playoff' column
label_encoder = LabelEncoder()
teams['playoff'] = label_encoder.fit_transform(teams['playoff'])
#teams['previous_playoff'] = label_encoder.fit_transform(teams['previous_playoff'])
teams['tmID'] = label_encoder.fit_transform(teams['tmID'])
teams['confID'] = label_encoder.fit_transform(teams['confID'])
#teams = pd.get_dummies(teams, columns=['tmID', 'confID'])


train_data = teams[teams['year'] <= 9]
test_data = teams[teams['year'] == 10]

X_train = train_data.drop(['playoff'], axis=1)
y_train = train_data['playoff']

X_test = test_data.drop(['playoff'], axis=1)
y_test = test_data['playoff']

clf = DecisionTreeClassifier(random_state=0)

# Fit the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test set
predictions = clf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7692307692307693


In [394]:
from sklearn.linear_model import LogisticRegression

# Use the same train_data and test_data from your code

X_train = train_data.drop(['playoff'], axis=1)
y_train = train_data['playoff']

X_test = test_data.drop(['playoff'], axis=1)
y_test = test_data['playoff']

# Initialize the logistic regression classifier
logreg_clf = LogisticRegression(solver='liblinear',random_state=0)

# Fit the classifier on the training data
logreg_clf.fit(X_train, y_train)

# Make predictions on the test set
logreg_predictions = logreg_clf.predict(X_test)

# Evaluate the accuracy of the model
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
print("Logistic Regression Accuracy:", logreg_accuracy)


Logistic Regression Accuracy: 0.8461538461538461


In [395]:
teams.to_csv('./Modified/modified_teams.csv', index=False)
merged_data.to_csv('./Modified/players_playersTeams.csv', index=False)
coaches.to_csv('./Modified/modified_coaches.csv', index=False)